<a href="https://colab.research.google.com/github/amkkashani/ChatRoom/blob/master/MapReducePySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing pyspark**

In [1]:
!pip install pyspark py4j

     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 199 kB 59.1 MB/s 
     |████████████████████████████████| 198 kB 53.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=9517a5c76a5c60dd98bae144b2da41ab5028236134ff242e35a9e4ab1e548a85
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from google.colab import drive


**Remove last sparkContex and make new one**




In [ ]:
from pyspark import SparkContext, SparkConf
sc = SparkContext.getOrCreate()
sc.stop()
sc = SparkContext("local", "count friend")

**Input Data**

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/bigData/q1/dataset1.txt", names = ["id" , "friends"],delimiter = "\t" )
df = df.dropna() # remove nulls rows
df["friends"] = df["friends"].apply(lambda inputRow :  list(inputRow.split(",")))


In [ ]:
# convert to dict for faster access
dictionaryByid = df.set_index('id').T.to_dict('records')
dataLists = df.to_dict('records')
# dictionaryByid[0][98]



### **MAP**

In [ ]:
rdd = sc.parallelize(dataLists)

def mapData(row):
  result = []
  myFriends = row["friends"]

  for i in range(len(myFriends)):
    semiFriendShip = {}
    
    for j in range(len(myFriends)):
      if i == j :
        continue  
      semiFriendShip[myFriends[j]] = 1
    
    result.append((myFriends[i], semiFriendShip))

  return result

mapResult = rdd.flatMap(mapData)  

### **Reduce**

In [ ]:
def reduceData(input1 , input2):
  keys1 = set(input1.keys())
  keys2 = set(input2.keys())

  kingKey = keys1 | keys2 # all of the keys

  freindshipPoints = {}
  for key in kingKey:
    friendShipPoint = 0

    v1 = input1.get(key)
    if v1 != None:
      friendShipPoint += v1
    
    v2 = input2.get(key)
    if v2 != None:
      friendShipPoint += v2

    freindshipPoints[key] = friendShipPoint

  return freindshipPoints

# usage
reduceResult = mapResult.reduceByKey(reduceData)
finallResult = reduceResult.collect()


In [ ]:
finallResult

In [ ]:
def findBestChoice(id, number_top):
  closeFriends = dictionaryByid[0][id]
  friendshipPoints = {}
  finalList = []

  # find the list of semifreinds
  for keys in finallResult:
    if keys[0] == id.__str__():
      friendshipPoints = keys[1]
      break

  # sort semi freinds
  rankingOfFreinds = {k: v for k, v in sorted(friendshipPoints.items(), key=lambda item: item[1],reverse=True)}

  

  for freind in rankingOfFreinds.keys():
    if freind in closeFriends:
      continue    
    finalList.append(freind)
    if len(finalList) == number_top:
      break

  return finalList

In [ ]:
for id in [98, 135, 117, 911, 8804]:
  print("id : " + id.__str__())
  print("suggested friends :" + findBestChoice(id,20).__str__())

id : 98
suggested friends :['18560', '16350', '30691', '13654', '2554', '16324', '30134', '12326', '21260', '2782', '41775', '40344', '44370', '12577', '21178', '2901', '26395', '47425', '6142', '2495']
id : 135
suggested friends :['13792', '33060', '629', '5490', '45054', '19217', '34441', '25256', '34164', '34151', '19358', '22357', '34250', '35613', '11399', '142', '19431', '12519', '3575', '442']
id : 117
suggested friends :['34164', '23507', '12519', '15314', '34207', '34220', '23510', '34140', '34169', '25256', '13793', '11399', '13867', '23615', '5490', '34462', '23513', '22340', '19095', '34390']
id : 911
suggested friends :['39540', '24456', '40560', '30995', '41352', '30993', '37875', '30984', '33333', '30996', '31056', '20191', '30952', '1698', '40639', '30957', '39491', '39526', '31001', '31028']
id : 8804
suggested friends :['34179', '34332', '8677', '11399', '13872', '29745', '34233', '15207', '3230', '34174', '11400', '13182', '13792', '19427', '34466', '34326', '623', '